<a href="https://colab.research.google.com/github/agalvezm/ACE2_scRNAseq/blob/master/tcc/GSE130888_GSM3755697_from_gc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GSE130888_GSM3755697

This notebook uses the filtered count matrix resulting from the following google colab notebook. https://github.com/agalvezm/ACE2_scRNAseq/blob/master/notebooks_countmatrices/GSE130888_GSM3755697.ipynb Please run the notebook above and upload the results file in /content

In [27]:
# define the values for the analysis

# accession id for the data
id = "GSE130888"
samp_id = ["GSM3755697"]

# If only bam available files, set bam = True, Fill link and filename
bam = False

# If fastq links available but are not ffq links
fastq_ffqlinks = False


if bam:
  # Assign link to python variable
  link_to_bam = ""
  
  # Assigns the link to the bash variable BAM_LINK. To be used by wget
  %env BAM_LINK=$link_to_bam

  # Assign filename to python variable. Used to convert to fastq and remove bam file to fastq after conversion

  bam_filename=""


if not bam:

  # Copy and paste the links from the ACE2 scRNAseq datasets google  spreadsheet
  links_raw = "ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR902/008/SRR9029608/SRR9029608_1.fastq.gz	ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR902/008/SRR9029608/SRR9029608_2.fastq.gz"
#  links_raw = "https://sra-pub-src-2.s3.amazonaws.com/SRR9029604/sampleQSL_S1_L001_R1_001.fastq.gz	https://sra-pub-src-2.s3.amazonaws.com/SRR9029604/sampleQSL_S1_L001_R2_001.fastq.gz"

  # Convert it to a list where each link is an element
  
  fastqs = links_raw.split()


In [28]:
no_samples = 1

fastqs_per_sample = [2] 

sample_id = samp_id

database_id = [id] * no_samples

tissue = ["peritoneal"] * no_samples

cell_type = ["peritoneal cells"] * no_samples

condition = ["long-term PD patient"] * no_samples

species = ["human"] * no_samples

technology = ["10xv2"] * no_samples

paper = ["Muus et al 2020"] * no_samples

figure = ["Fig 1 a,b  ED Fig 1 a,b,c,d  ED Fig 2 a,b,c,d,e"] * no_samples


# Set string variables for kb functions

species_kb = species[0]

technology_kb = technology[0]

# Imports and installs

In [29]:
# install and import necessary software

# Install kb and scanpy
!pip -q install kb-python 
!pip -q install scanpy

import re
import os

# Setup

import anndata
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as mpatches
import scanpy as sc
from scipy import stats

from collections import OrderedDict
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE
from sklearn.preprocessing import scale

from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import NeighborhoodComponentsAnalysis
from matplotlib import cm
from matplotlib.lines import Line2D

def nd(arr):
    return np.asarray(arr).reshape(-1)
def yex(ax):
    lims = [np.min([ax.get_xlim(), ax.get_ylim()]),
            np.max([ax.get_xlim(), ax.get_ylim()])]

    # now plot both limits against eachother
    ax.plot(lims, lims, 'k-', alpha=0.75, zorder=0)
    ax.set_aspect('equal')
    ax.set_xlim(lims)
    ax.set_ylim(lims)
    return ax

def trim_axs(axs, N):
    """little helper to massage the axs list to have correct length..."""
    axs = axs.flat
    for ax in axs[N:]:
        ax.remove()
    return axs[:N]

import warnings
warnings.filterwarnings('ignore')

fsize=20

plt.rcParams.update({'font.size': fsize})
%config InlineBackend.figure_format = 'retina'

# Downloads: (bam (if bam) and index

In [30]:
if bam:

  # Install bamtofastq from 10x website (only bam files available)
  !wget http://cf.10xgenomics.com/misc/bamtofastq-1.2.0
  !chmod +x bamtofastq-1.2.0
  # Download the bam file
  !wget -- continue ${BAM_LINK}




In [31]:
if bam:
  # Convert to fastq
  !./bamtofastq-1.2.0 --reads-per-fastq=500000000 $bam_filename ./fastqs\

  # Remove original bam file to save space
  !rm $bam_filename


In [32]:
# Store fastq names on a list

if bam:
  # cd into fastqs folder
  %cd /content/fastqs

  #store the name of the folder generated by bamtofastq
  _filename = os.listdir()[0]

  # cd into that folder
  %cd $_filename

  # store fastq names in a list
  fastqs = os.listdir()


  # Remove I1 and R3 reads not relevant for our analysis

  # Initialize list containing elements to remove
  remov_elem = []

  print ("\n\nThis is the complete list of fastqs:\n -----------")
  for elem in fastqs:
    print (elem)

  # Search index (I1 or R3) fastqs and remove them from list
  for elem in fastqs:
    if re.search("_R3_", elem) or re.search("_I1_", elem):
      remov_elem = remov_elem +[elem]

  fastqs = [elem for elem in fastqs if elem not in remov_elem] 

  print ("\n\nThis is the filtered list of fastqs:\n -----------")
  for elem in fastqs:
    print (elem)


In [33]:
# Remove fastqs that wont be analyzed to save space
if bam:
  for elem in remov_elem:
    !rm $elem

In [34]:
if bam:
  # sort fastqs alphabetically to get R1 and R2 in order
  fastqs = sorted(fastqs)

In [35]:
# wget fastqs from non ffq links in fastqs folder
if not bam and not fastq_ffqlinks:
  !mkdir fastqs
  %cd fastqs
  for link in fastqs:
    !wget $link --continue

  # update fastqs variable with name of files
  fastqs = sorted(os.listdir())

mkdir: cannot create directory ‘fastqs’: File exists
/content/fastqs
--2021-03-09 01:02:47--  ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR902/008/SRR9029608/SRR9029608_1.fastq.gz
           => ‘SRR9029608_1.fastq.gz’
Resolving ftp.sra.ebi.ac.uk (ftp.sra.ebi.ac.uk)... 193.62.193.138
Connecting to ftp.sra.ebi.ac.uk (ftp.sra.ebi.ac.uk)|193.62.193.138|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /vol1/fastq/SRR902/008/SRR9029608 ... done.
==> SIZE SRR9029608_1.fastq.gz ... 5535081446
File has already been retrieved.
2021-03-09 01:02:49 (0.00 B/s) - ‘SRR9029608_1.fastq.gz’ saved [5535081446]

--2021-03-09 01:02:49--  ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR902/008/SRR9029608/SRR9029608_2.fastq.gz
           => ‘SRR9029608_2.fastq.gz’
Resolving ftp.sra.ebi.ac.uk (ftp.sra.ebi.ac.uk)... 193.62.193.138
Connecting to ftp.sra.ebi.ac.uk (ftp.sra.ebi.ac.uk)|193.62.193.138|:21... connected.
Logging in as anonymous ... Logg

In [36]:
if bam:
  # Download the corresponding Kallisto index to folder containing fastqs
  !kb ref -d $species_kb -i index.idx -g t2g.txt -f1 transcriptome.fasta

if not bam and fastq_ffqlinks:
  %cd /content

  # Download the corresponding Kallisto index to content folder
  !kb ref -d $species_kb -i index.idx -g t2g.txt -f1 transcriptome.fasta

if not bam and not fastq_ffqlinks:
  %cd /content/fastqs

  # Download the corresponding Kallisto index to fastq folder
  !kb ref -d $species_kb -i index.idx -g t2g.txt -f1 transcriptome.fasta

/content/fastqs
[2021-03-09 01:02:53,070]    INFO Downloading files for human from https://caltech.box.com/shared/static/v1nm7lpnqz5syh8dyzdk2zs8bglncfib.gz to tmp/v1nm7lpnqz5syh8dyzdk2zs8bglncfib.gz
100% 2.23G/2.23G [01:36<00:00, 24.8MB/s]
[2021-03-09 01:04:30,788]    INFO Extracting files from tmp/v1nm7lpnqz5syh8dyzdk2zs8bglncfib.gz


In [37]:
# Check to make sure the metadata is in the right order after sorting
print(fastqs)

['SRR9029608_1.fastq.gz', 'SRR9029608_2.fastq.gz', 'index.idx', 'outputGSM3755697', 't2g.txt']


# Process fastq files (modify kb command according to fastqs list)


In [38]:
fastqs = ['SRR9029608_1.fastq.gz', 'SRR9029608_2.fastq.gz']

In [39]:

# Specify the sample number and whether they are paired-ended


if no_samples == 1:

  # Write the kb count command as a string with all fastqs of the list as an input
  cmd = "kb count --h5ad -i index.idx -g t2g.txt -x " + technology_kb + " -o tccoutput" + sample_id[0] + " "\
  + "--filter bustools --tcc -t 2 --overwrite " + "'" +  "' '".join(fastqs) + "'"
  
  # Execute it
  !$cmd

# If more than one sample, iterate through fastqs accordingly
else:

  # Initializa counter for fastq files
  j = 0

  # Loop over samples for analysis
  for i in range(no_samples):

    fastqs_to_analyze = fastqs[j:j + fastqs_per_sample[i]]
    # Write the kb count command as a string
    cmd = "kb count --h5ad -i ../index.idx -g ../t2g.txt -x " + technology_kb + " -o tccoutput" + sample_id[i] + " \
    --filter bustools --tcc -t 2 --overwrite " + "'" +  "' '".join(fastqs_to_analyze) + "'"

    # Execute it
    !$cmd

    # Update j to move to the next set of fastq
    j = j + fastqs_per_sample[i]






results = {}
tcc_results = {}
for sample in sample_id:
  # Read the filtered gene count matrix
  results["data_" + sample] = anndata.read("/content/result" + sample)


for sample in sample_id:
  output = "bus_raw_" + sample + "_from_gc"
  !mkdir $output
  folder_tcc = "tccoutput" + sample

  # read tcc matrix
  tcc_results[sample] = anndata.read(folder_tcc + "/counts_unfiltered/adata.h5ad")
  # apply gene count matrix's filter
  tcc_results[sample] = tcc_results[sample][results["data_" + sample].obs.index.values]
  # transfer obs data
  tcc_results[sample].obs = results["data_" + sample].obs
  # transfer metadata
  tcc_results[sample].uns = results["data_" + sample].uns
  # write tcc matrix
  tcc_results[sample].write("/content/tcc_" + sample)
  #gzip tcc matrix
  cmd = "gzip /content/tcc_" + sample
  !$cmd
  #gzip unfiltered bus file
  cmd = "gzip " + folder_tcc + "/output.unfiltered.bus"
  !$cmd
  # move and re-name bus file with sample id
  cmd = "mv " + folder_tcc + "/output.unfiltered.bus.gz " + output + "/" + sample +".unfiltered.bus.gz"
  !$cmd
  # gzip all unfiltered counts
  cmd = "gzip " + folder_tcc + "/counts_unfiltered/*"
  !$cmd

  # move unfiltered counts for both
  cmd = "mv " + folder_tcc + "/counts_unfiltered " + output + "/tcc_unfiltered"
  !$cmd

  cmd = "mv " + folder_tcc + "/*.json " + output 
  !$cmd
  # zip all files
  cmd = "zip -r "+ output + ".zip " + output
  !$cmd

[2021-03-09 01:05:23,855]    INFO Using index index.idx to generate BUS file to tccoutputGSM3755697 from
[2021-03-09 01:05:23,856]    INFO         SRR9029608_1.fastq.gz
[2021-03-09 01:05:23,856]    INFO         SRR9029608_2.fastq.gz
[2021-03-09 02:14:29,185]    INFO Sorting BUS file tccoutputGSM3755697/output.bus to tccoutputGSM3755697/tmp/output.s.bus
[2021-03-09 02:17:13,086]    INFO Whitelist not provided
[2021-03-09 02:17:13,087]    INFO Copying pre-packaged 10XV2 whitelist to tccoutputGSM3755697
[2021-03-09 02:17:13,225]    INFO Inspecting BUS file tccoutputGSM3755697/tmp/output.s.bus
[2021-03-09 02:17:49,527]    INFO Correcting BUS records in tccoutputGSM3755697/tmp/output.s.bus to tccoutputGSM3755697/tmp/output.s.c.bus with whitelist tccoutputGSM3755697/10xv2_whitelist.txt
[2021-03-09 02:18:16,505]    INFO Sorting BUS file tccoutputGSM3755697/tmp/output.s.c.bus to tccoutputGSM3755697/output.unfiltered.bus
[2021-03-09 02:19:32,351]    INFO Generating count matrix tccoutputGSM3755

Variable names are not unique. To make them unique, call `.var_names_make_unique`.


  adding: bus_raw_GSM3755697_from_gc/ (stored 0%)
  adding: bus_raw_GSM3755697_from_gc/kb_info.json (deflated 75%)
  adding: bus_raw_GSM3755697_from_gc/inspect.json (deflated 57%)
  adding: bus_raw_GSM3755697_from_gc/tcc_unfiltered/ (stored 0%)
  adding: bus_raw_GSM3755697_from_gc/tcc_unfiltered/adata.h5ad.gz (deflated 6%)
  adding: bus_raw_GSM3755697_from_gc/tcc_unfiltered/cells_x_tcc.barcodes.txt.gz (deflated 1%)
  adding: bus_raw_GSM3755697_from_gc/tcc_unfiltered/cells_x_tcc.mtx.gz (deflated 0%)
  adding: bus_raw_GSM3755697_from_gc/tcc_unfiltered/cells_x_tcc.ec.txt.gz (deflated 2%)
  adding: bus_raw_GSM3755697_from_gc/run_info.json (deflated 38%)
  adding: bus_raw_GSM3755697_from_gc/GSM3755697.unfiltered.bus.gz (deflated 4%)
